In [7]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# Load data
dataset = pd.read_csv('../../A4-Apriori/basket.csv' )
dataset.head()

,Unnamed: 0,Invoice,Description,Quantity,InvoiceDate,period_day,Weekend,DayOfWeek,Day,MonthName,Month,Hour,HourInterval
0,0,489434,15cm christmas glass ball 20 lights,12,2009-12-01 07:45:00,Morning,Weekday,Tuesday,Tuesday,December,December,7,7-8
1,1,489434,pink cherry lights,12,2009-12-01 07:45:00,Morning,Weekday,Tuesday,Tuesday,December,December,7,7-8
2,2,489434,white cherry lights,12,2009-12-01 07:45:00,Morning,Weekday,Tuesday,Tuesday,December,December,7,7-8
3,3,489434,"record frame 7"" single size",48,2009-12-01 07:45:00,Morning,Weekday,Tuesday,Tuesday,December,December,7,7-8
4,4,489434,strawberry ceramic trinket box,24,2009-12-01 07:45:00,Morning,Weekday,Tuesday,Tuesday,December,December,7,7-8


In [9]:
# Group products by transaction
grouped_prods = dataset.groupby('Invoice')['Description'].apply(lambda group_series: group_series.tolist()).reset_index()
groups_lists = grouped_prods['Description'].values.tolist()
# Convert table to list
# Set threshold of count to 2
data = list(filter(lambda x: len(x) > 2, groups_lists))

In [10]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,*boombox ipod classic,*usb office glitter lamp,*usb office mirror ball,10 colour spaceboy pen,11 pc ceramic tea set polkadot,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18992,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18993,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18994,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
#use fp-growth algorithm
from mlxtend.frequent_patterns import fpgrowth

f_patterns = fpgrowth(df, min_support=0.005, use_colnames=True)
f_patterns

,support,itemsets
0,0.089646,(strawberry ceramic trinket box)
1,0.025372,(pink doughnut trinket pot)
2,0.024004,(save the planet mug)
3,0.021477,(white cherry lights)
4,0.016739,(pink cherry lights)
...,...,...
7399,0.007528,"(hand warmer union jack, hand warmer owl desig..."
7400,0.007685,"(hand warmer owl design, hand warmer red retro..."
7401,0.006475,"(hand warmer union jack, hand warmer red retro..."
7402,0.006948,"(hand warmer scotty dog design, hand warmer re..."


In [12]:
import itertools as it
from itertools import *

# help function
def partition(pred, iterable):
    t1, t2 = it.tee(iterable)
    return it.filterfalse(pred, t1), filter(pred, t2)

# divides list on all possible pairs
def part2(el_list):
    pairs = [[[x[1] for x in f] for f in partition(lambda x: x[0], zip(pattern, el_list))] 
    for pattern in product([True, False], repeat=len(el_list))]
    # remove pairs as [] -> [some content], [some content] -> []
    return pairs[1:-1]


In [13]:
# convert dataframe to dictionary
supports = f_patterns['support'].to_list()
itemsets = f_patterns['itemsets'].to_list()

patterns_dict = {}
for x in range(len(itemsets)):
    patterns_dict[tuple(sorted(itemsets[x]))] = supports[x]
    
# generate asssociation_rules
as_rules_dict = {'left': [], 'right': [], 'confidence': []}
for pattern, support in patterns_dict.items():
    if len(pattern) > 1:
        upper_support = support
        as_rules = part2(pattern)
        
        for as_r in as_rules:
            left_part = sorted(as_r[0])
            right_part = as_r[1]
            lower_support = patterns_dict[tuple(left_part)]
            conf = upper_support / lower_support            
            as_rules_dict['left'].append(left_part)
            as_rules_dict['right'].append(right_part)
            as_rules_dict['confidence'].append(conf)
            

strong_as_rules = pd.DataFrame.from_dict(as_rules_dict)
# sort by confidence, remove all rules with confidence lower than 0.8
strong_as_rules = strong_as_rules.sort_values('confidence', ascending=False)
strong_as_rules = strong_as_rules[strong_as_rules['confidence'] > 0.8]

strong_as_rules

,left,right,confidence
16755,"[childrens garden gloves blue, childs garden f...",[childs garden trowel pink],1.000000
16923,"[childrens garden gloves blue, childrens garde...",[childs garden trowel pink],1.000000
16493,"[childrens garden gloves blue, childrens garde...",[childs garden trowel pink],1.000000
16623,"[childrens garden gloves blue, childrens garde...",[childs garden trowel pink],1.000000
16579,"[childrens garden gloves blue, childs garden f...",[childs garden trowel pink],1.000000
...,...,...,...
12009,"[72 sweetheart fairy cake cases, dotcom postage]",[strawberry ceramic trinket box],0.801370
1955,"[heart of wicker large, red hanging heart t-li...",[white hanging heart t-light holder],0.801242
5697,"[green spotty cup, pink spotty cup]","[blue spotty cup, red spotty cup]",0.801170
16222,[monsters stencil craft],[happy stencil craft],0.801047


In [14]:
strong_as_rules.to_csv('OutPut.csv',index=True)
print("Done")

Done
